In [11]:
import torch

print(torch.version.cuda)
print(torch.cuda.get_device_name(0))

12.8
NVIDIA GeForce RTX 4060 Laptop GPU


In [12]:
allocated_bytes = torch.cuda.memory_allocated('cuda')
allocated_mb = allocated_bytes / (1024 * 1024)

reserved_bytes = torch.cuda.memory_reserved('cuda')
reserved_mb = reserved_bytes / (1024 * 1024)

print("\n--- VRAM USAGE REPORT ---")
print(f"Memory Allocated: {allocated_mb:.2f} MB")
print(f"Memory Reserved:  {reserved_mb:.2f} MB")
print("-------------------------\n")


--- VRAM USAGE REPORT ---
Memory Allocated: 0.00 MB
Memory Reserved:  0.00 MB
-------------------------



In [27]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

validation_pipeline = A.Compose([
    A.Resize(height=224, width=224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

basic_pipeline = A.Compose([
    # Resize the image first
    A.Resize(height=224, width=224),
    A.RandomCrop(height=180, width=180, p=0.7),
    A.Resize(height=224, width=224),
    A.HorizontalFlip(p=0.6),
    A.VerticalFlip(p=0.6),
    A.Rotate(limit=(-45,45), p=0.7),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [22]:
import torch.nn as nn
from torchvision import models

class PestClassifierResNet50(nn.Module):

    def __init__(self, num_classes):

        super(PestClassifierResNet50,self).__init__()

        self.base_model = models.resnet50(weights=None)

        features = self.base_model.fc.in_features

        self.base_model.fc = nn.Sequential(
            nn.Dropout(p=0.3, inplace=False),
            nn.Linear(features, num_classes),
        )

    def forward(self, x):

        return self.base_model(x)

In [4]:

model = models.resnet50(weights=None)

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [28]:
import torch
from tqdm import tqdm
import time

def train_and_validate(model, optimizer, criterion, train_dataloader, validation_dataloader, num_epoch, device):

    total_time = 0.0
    train_losses = []
    validation_losses = []
    accuracies = []
    best_validation_loss=float('inf')

    start_time = time.time()

    for epoch in range(num_epoch):
        model.train()
        running_train_loss = 0.0
        train_pbar = tqdm(train_dataloader, desc=f"Train epoch: {epoch+1}")
        for data, labels in train_pbar:
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()
        training_loss = running_train_loss / len(train_dataloader)
        train_losses.append(training_loss)

        model.eval()
        running_eval_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        with torch.no_grad():
            for inputs, labels in tqdm(validation_dataloader, desc=f"Validate Epoch {epoch+1}"):
                inputs, labels = inputs.to(device), labels.to(device)

                logits = model(inputs)
                loss = criterion(logits, labels)
                running_eval_loss += loss.item()
                _, predicted = torch.max(logits.data, 1)
                total_samples += labels.size(0)
                correct_predictions += (predicted == labels).sum().item()

        validation_loss = running_eval_loss / len(validation_dataloader)
        validation_losses.append(validation_loss)
        accuracy = 100 * correct_predictions / total_samples
        accuracies.append(accuracy)

        epoch_summary = f'Epoch {epoch+1}/{num_epoch} | Train Loss: {training_loss:.4f} | Evaluate Loss: {validation_loss:.4f} | Accuracy: {accuracy:.2f}%'

        if device.type == 'cuda':
            allocated_gb = torch.cuda.memory_allocated() / (1024 ** 3)
            reserved_gb = torch.cuda.memory_reserved() / (1024 ** 3)
            epoch_summary += f' | GPU Memory (Alloc/Reserved): {allocated_gb:.2f}/{reserved_gb:.2f} GB'

        if validation_loss < best_validation_loss - 0.001:

            best_validation_loss = validation_loss
            torch.save(model.state_dict(), 'MobileNetv2_result/models/model1.pt')

        tqdm.write(epoch_summary)

        end_time = time.time()
        total_time = end_time - start_time

    return  {'train_loss': train_losses, 'validation_loss': validation_losses, 'accuracy': accuracies}, total_time

In [29]:
from torch.utils.data import DataLoader
from helpers import CustomDataset, plot_training_result, plot_accuracy, save_historical_data

train_set = CustomDataset(root='datasets/train', transform=basic_pipeline)
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
validation_set = CustomDataset(root='datasets/val', transform=validation_pipeline)
validation_loader = DataLoader(validation_set, batch_size=4, shuffle=True)
test_set = CustomDataset(root='datasets/test', transform=validation_pipeline)
test_loader = DataLoader(test_set, batch_size=4, shuffle=True)

classes = train_set.classes

<h2> Hyper parameter tuning</h2>

In [30]:
from torch.optim import Adam

hardware = torch.device('cuda' if  torch.cuda.is_available() else 'cpu')
classes = 6
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=1e-4)

model = PestClassifierResNet50(num_classes=classes).to(hardware)

result, time = train_and_validate(model, optimizer, criterion, train_loader, validation_loader, num_epoch=16, device=hardware)

print(f"Training time: {time:.2f}")

plot_training_result(*result)
plot_accuracy(*result)
save_historical_data(*result, filename='MobileNetv2_result/hyper_parameter_tuning/model1.csv')

Validate Epoch 1: 100%|██████████| 676/676 [00:13<00:00, 49.79it/s]


Epoch 1/16 | Train Loss: 2.0373 | Evaluate Loss: 1.9941 | Accuracy: 12.54% | GPU Memory (Alloc/Reserved): 0.30/1.87 GB


Validate Epoch 2: 100%|██████████| 676/676 [00:10<00:00, 64.73it/s]


Epoch 2/16 | Train Loss: 2.0439 | Evaluate Loss: 1.9926 | Accuracy: 11.21% | GPU Memory (Alloc/Reserved): 0.30/1.87 GB


Validate Epoch 3: 100%|██████████| 676/676 [00:10<00:00, 64.43it/s]


...

Validate Epoch 15: 100%|██████████| 676/676 [00:10<00:00, 64.50it/s]


Epoch 15/16 | Train Loss: 2.0368 | Evaluate Loss: 2.0079 | Accuracy: 12.02% | GPU Memory (Alloc/Reserved): 0.30/1.87 GB


Validate Epoch 16: 100%|██████████| 676/676 [00:10<00:00, 62.47it/s]


Epoch 16/16 | Train Loss: 2.0413 | Evaluate Loss: 1.9887 | Accuracy: 12.20% | GPU Memory (Alloc/Reserved): 0.30/1.87 GB
Training time: 1451.28


AttributeError: 'str' object has no attribute 'get'